# Init

In [1]:
import time, sys, os
%load_ext autoreload
%autoreload 2

In [2]:
import pickle as pk

## Custom tools

In [3]:
sys.path.insert(0, "/home/pleroy/DEV/processing/PoSAR-MC")
import posarmctools.readdata as readdata

In [4]:
sys.path.insert(0, "/home/pleroy/DEV/processing/focalization_python")
import posarutils.process.disp_PoSAR_img as disp

In [5]:
sys.path.insert(0, "/home/pleroy/DEV/DIADEM")
import toolsdiadem.tools as dia
import toolsdiadem.posargb as gb

In [6]:
i_CFG = 0
i_RD = 1
i_CAL = 2
i_f = 3

i_x = 2
i_y = 3
i_z = 4

# antenna locations on the plate
Dx1   = 0.224
Dx2   = 0.168
h     = 0.04
# positions of the box in x
xMinTrail  = -1.50
xMaxTrail  = 1.493751
# bizona
x_biz_offset = 0
y_biz_offset = 6.7
z_biz_offset = 3
biz_offset = (x_biz_offset, y_biz_offset, z_biz_offset)
# delay
#delays = {'1':0.1, '2':0.12, '3':0.12, '4':0.25}
delays = {'1':0.1, '2':0.1, '3':0.1, '4':0.1}

In [7]:
from backprojection.loadbackprojection import *
libraryFilename = "/home/pleroy/DEV/processing/PoSAR-MC/backprojection/ccpp/libbackprojection/liblibbackprojection.so"
lib = LibBackProjection( libraryFilename )

## Numpy

In [8]:
import numpy as np
import matplotlib.pyplot as plt

In [9]:
%matplotlib qt5

## Data paths

In [10]:
date = "2019_03_04"
base_path = "/home/pleroy/DATA/DIADEM/lff/Bizona/" + date
prefix = "sand_trunk_vol"
nbDir = 11
RxTx = ["21", "31", "32", "41", "42", "43"]

In [11]:
heights = {}
for k in range(1, nbDir+1):
    filename = base_path + f"/{prefix}_{k}/PoSAR.cfg"
    CFG = gb.readCfg( filename )
    heights[f"{k}"] = CFG["H_nom"]

In [12]:
heights

{'1': 3.41,
 '2': 3.33,
 '3': 3.25,
 '4': 3.17,
 '5': 3.09,
 '6': 3.01,
 '7': 2.93,
 '8': 2.85,
 '9': 2.77,
 '10': 2.69,
 '11': 2.61}

# Read separate files

## Read configuration

In [13]:
CFG = gb.readCfg( base_path + f"/{prefix}_1/PoSAR.cfg")

In [14]:
CFG

{'H_nom': 3.41,
 'xamin_nom': -1.5,
 'xamax_nom': 1.493751,
 'Npos': 500,
 'theta_c': 0.7853981633974483,
 'theta_ap': 0.7853981633974483,
 'phi_c': 0.0,
 'phi_ap': 0.7853981633974483,
 'Fmin': 8199999999.999999,
 'Fmax': 12400000000.0,
 'Nf': 4001,
 'c': 300000000.0}

In [15]:
Npos = CFG["Npos"]

## Read positions

In [16]:
data_idx = 1
ij = "41"

In [17]:
data_dir = f"{prefix}_{data_idx}"
ant_pos = gb.readPos( base_path + f"/{data_dir}/track_S{ij}.dat" )

In [22]:
fig, ax = plt.subplots(1,1)
ax.plot(ant_pos.Txx, '.-', label=f"Txx")
ax.plot(ant_pos.Txy, '.-', label=f"Txy")
ax.plot(ant_pos.Txz, '.-', label=f"Txz")
ax.plot(ant_pos.Rxx, '.-', label=f"Rxx")
ax.plot(ant_pos.Rxy, '.-', label=f"Rxy")
ax.plot(ant_pos.Rxz, '.-', label=f"Rxz")
ax.legend()
ax.grid()
title = data_dir + f"\nS{ij}"
ax.set_title(title)

Text(0.5, 1.0, 'sand_trunk_vol_1\nS41')

## Read data

In [19]:
data_idx = 1
ij = "41"

In [20]:
z3 = heights[f"{data_idx}"]
data = gb.readData( f"{base_path}/{prefix}_{data_idx}", f"S{ij}.dat" )

In [21]:
f_base = data[i_f] - data[i_CFG]["Fmin"]
r = np.arange(f_base.size) * data[i_CFG]["delta_sr"]
az = (ant_pos.Txx + ant_pos.Txz)/2

In [23]:
coupling_RD = np.average( data[i_RD], 0 )
ifft_RD = np.fft.ifft( data[i_RD] )

In [24]:
fig, ax = plt.subplots(1,1)

im = ax.plot( r, 20 * np.log10( np.abs( coupling_RD ) ), label='coupling' )
ix = int( Npos / 2 )
im = ax.plot( r, 20 * np.log10( np.abs( ifft_RD[ix,:] ) ), label=f'[{ij}] ifft_RD z3={z3} ix={ix}' )
ix = 0
im = ax.plot( r, 20 * np.log10( np.abs( ifft_RD[ix,:] ) ), label=f'[{ij}] ifft_RD z3={z3} ix={ix}' )
ix = -1
im = ax.plot( r, 20 * np.log10( np.abs( ifft_RD[ix,:] ) ), label=f'[{ij}] ifft_RD z3={z3} ix={ix}' )
ax.grid()
ax.legend()

In [25]:
ifft_RD.shape

(500, 4001)

In [26]:
fig, (ax0, ax1) = plt.subplots(1,2,sharex=True,sharey=True)

im = ax0.pcolormesh( r, az, 20 * np.log10( np.abs( ifft_RD ) ), cmap='jet' )
dia.addColorBar(im, ax0, 'auto')

im = ax1.pcolormesh( r, az, np.angle( ifft_RD ), cmap='jet' )
dia.addColorBar(im, ax1, 'auto')

fig.suptitle( f"{date} {prefix} [{ij}] z3={z3} raw data" )

Text(0.5, 0.98, '2019_03_04 sand_trunk_vol [41] z3=3.41 raw data')

# Build positions

In [27]:
data_idx = 1
#data_path = base_path + f"/Volume_sol_{data_idx}"
data_path = base_path + f"/sand_trunk_vol_{data_idx}"
Npos  = CFG["Npos"]
zRail = heights[f"{data_idx}"]
xyz_dic = gb.buildPos( xMinTrail, xMaxTrail, Npos, Dx1, Dx2, h, zRail )
xyz_dicUsingTrack = gb.buildPosUsingTrack( data_path + "/track_S" , Dx1, Dx2, h, zRail, verbose=1, bizona=biz_offset )
ant_pos21 = gb.readPos( data_path + "/track_S21.dat" )
ant_pos43 = gb.readPos( data_path + "/track_S43.dat" )

read 500 positions in file:
/home/pleroy/DATA/DIADEM/lff/Bizona/2019_03_04/sand_trunk_vol_1/track_S21.dat


In [31]:
fig, ax = plt.subplots(1,1)
for a in ["1", "2", "3", "4"]:
    ax.plot( xyz_dic[a][:,i_x], xyz_dic[a][:,i_z], 'o', label=f"antenna {a}, xyz_dic" )
    ax.plot( xyz_dicUsingTrack[a][:,i_x], xyz_dicUsingTrack[a][:,i_z], 'x', label=f"antenna {a}, xyz_dicUsingTrack" )
ax.plot( ant_pos21.Txx, ant_pos21.Txz, '.', label="antenna 1, ant_pos read positions" )
ax.plot( ant_pos21.Rxx, ant_pos21.Rxz, '.', label="antenna 2, ant_pos read positions" )
ax.plot( ant_pos43.Txx, ant_pos43.Txz, '.', label="antenna 3, ant_pos read positions" )
ax.plot( ant_pos43.Rxx, ant_pos43.Rxz, '.', label="antenna 4, ant_pos read positions" )
ax.legend()
ax.grid()
ax.set_title( f"positions computed" )

Text(0.5, 1.0, 'positions computed')

In [29]:
fig, ax = plt.subplots(1,1)
a = "3"
ax.plot( xyz_dicUsingTrack[a][:,i_x], 'o', label=f"antenna {a}, xyz_dicUsingTrack" )
ax.plot( xyz_dicUsingTrack[a][:,i_y], 'o', label=f"antenna {a}, xyz_dicUsingTrack" )
ax.plot( xyz_dicUsingTrack[a][:,i_z], 'o', label=f"antenna {a}, xyz_dicUsingTrack" )
ax.legend()
ax.grid()
ax.set_title( f"positions computed" )

Text(0.5, 1.0, 'positions computed')

In [32]:
fig, ax = plt.subplots(1,1)
for a in ["1", "2", "3", "4"]:
    ax.plot( xyz_dicUsingTrack[a][:,i_x], 'o', label=f"X antenna {a}, xyz_dicUsingTrack" )
    ax.plot( xyz_dicUsingTrack[a][:,i_y], 'o', label=f"Y antenna {a}, xyz_dicUsingTrack" )
    ax.plot( xyz_dicUsingTrack[a][:,i_z], 'o', label=f"Z antenna {a}, xyz_dicUsingTrack" )
ax.legend()
ax.grid()
ax.set_title( f"positions computed" )

Text(0.5, 1.0, 'positions computed')

In [33]:
fig, ax = plt.subplots(1,1)
ax.plot( ant_pos.Txz, 'o', label="antenna 1, ant_pos read positions" )
ax.legend()
ax.grid()
ax.set_title( f"positions computed" )

Text(0.5, 1.0, 'positions computed')

In [34]:
#path_1 = base_path + "/Volume_sol_1"
#path_2 = base_path + "/Volume_sol_2"
#path_3 = base_path + "/Volume_sol_3"

path_1 = base_path + "/sand_trunk_vol_1"
path_2 = base_path + "/sand_trunk_vol_2"
path_3 = base_path + "/sand_trunk_vol_3"

ant_pos_S21_1 = gb.readPos( path_1 + "/track_S21.dat" )
ant_pos_S21_2 = gb.readPos( path_2 + "/track_S21.dat" )
ant_pos_S21_3 = gb.readPos( path_3 + "/track_S21.dat" )

z1 = heights["1"]
xyz_1 = gb.buildPosUsingTrack( path_1 + "/track_S" , Dx1, Dx2, h, z1 )
z1 = heights["2"]
xyz_2 = gb.buildPosUsingTrack( path_2 + "/track_S" , Dx1, Dx2, h, z1 )
z1 = heights["3"]
xyz_3 = gb.buildPosUsingTrack( path_3 + "/track_S" , Dx1, Dx2, h, z1 )

In [35]:
fig, ax = plt.subplots(1,1)

ax.plot( ant_pos_S21_1.Txx, ant_pos_S21_1.Txz, '.', label="antenna 1, ant_pos_1" )
ax.plot( ant_pos_S21_2.Txx, ant_pos_S21_2.Txz, '.', label="antenna 1, ant_pos_2" )
ax.plot( ant_pos_S21_3.Txx, ant_pos_S21_3.Txz, '.', label="antenna 1, ant_pos_3" )

ax.plot( xyz_1["1"][:,i_x], xyz_1["1"][:,i_z], 'x', label="antenna 1, xyz_1" )
ax.plot( xyz_2["1"][:,i_x], xyz_2["1"][:,i_z], 'x', label="antenna 1, xyz_2" )
ax.plot( xyz_3["1"][:,i_x], xyz_3["1"][:,i_z], 'x', label="antenna 1, xyz_3" )

ax.legend()
ax.grid()
ax.set_title( f"positions computed" )

Text(0.5, 1.0, 'positions computed')

# Focalization

In [36]:
lib.reload()

## Define geometry

In [37]:
hScene = -1

dx = 0.01
dy = 0.01
dz = 0.01
xMin = -0.2
xMax = -0.2
yMin = 0
yMax = 7
zMin = -0.5
zMax = 1.5

x = np.arange(xMin, xMax+1e-9, dx)
y = np.arange(yMin, yMax+1e-9, dy)
z = np.arange(zMin, zMax+1e-9, dz)
idx0_x = int( np.size(x)/2 )
idx0_y = int( np.size(y)/2 )
idx0_z = int( np.size(z)/2 )
nbX = x.size
nbY = y.size
nbZ = z.size

# lefg right bottom top
extent_yz = [yMin, yMax, zMin, zMax]
extent_yx = [yMin, yMax, xMin, xMax]

In [38]:
x.shape, y.shape, z.shape

((1,), (701,), (201,))

In [39]:
print( f"center of the scene ( {np.mean(x):.2f}, {np.mean(y):.2f}, {np.mean(z):.2f} )")

center of the scene ( -0.20, 3.50, 0.50 )


## Backprojection

In [40]:
data_idx = 1
path_xxx = base_path + f"/{prefix}_{data_idx}"
ij = "41"
shiftI = delays[ij[0]]
shiftJ = delays[ij[1]]
d_shift = shiftI + shiftJ
overSamplingRatio = 10

read = gb.readData( path_xxx, f"S{ij}.dat" )
sr = np.fft.ifftshift( read[i_RD][:,0:read[i_CFG]["Nf"]-1], axes=1 )
rangeResolution = read[i_CFG]["delta_sr"]
Naz = sr.shape[0]
Nf  = sr.shape[1]

Rx, Tx = ij[0], ij[1]
z1 = heights[f"{data_idx}"]
xyz_dic = gb.buildPosUsingTrack( path_xxx + "/track_S" , Dx1, Dx2, h, z1, bizona=biz_offset )
xyzAntI = xyz_dic[ Rx ]
xyzAntJ = xyz_dic[ Tx ]

Nover   = overSamplingRatio * Nf
r_base  = np.arange( Nf ) * rangeResolution - d_shift
r_over  = np.arange( Nover ) * rangeResolution / overSamplingRatio - d_shift
dr_over = r_over[1] - r_over[0]

print( f"Nf = {Nf}, Naz = {Naz}, delay = {d_shift}" )
print( f"range from {r_over[0]:.2f}m to {r_over[-1]:.2f}m, "
      + f"resolution = {rangeResolution:.3e}m, oversampled = {rangeResolution / overSamplingRatio:.3e}m" )

Nf = 4000, Naz = 500, delay = 0.2
range from -0.20m to 142.65m, resolution = 3.571e-02m, oversampled = 3.571e-03m


In [56]:
xyzAntI.shape

(500, 5)

In [41]:
myParameters           = MyParametersPoSAR_GB()
myParameters.Nx        = nbX
myParameters.Ny        = nbY
myParameters.Nz        = nbZ
myParameters.Nover     = r_over.size
myParameters.dx        = dr_over
myParameters.Naz       = Naz
myParameters.Nf        = Nf
myParameters.phi_a_deg = 60 # 20° => 7.4cm
myParameters.kc        = read[i_CFG]["kc"]

In [42]:
t = time.time()

focusedImg_a = np.zeros( (nbX, nbY, nbZ), dtype=complex )
print( "focusedImg_a.shape = {}".format( focusedImg_a.shape ) )

lib.so.backProjectionOmpGroundRange_PoSAR_GB_a( x,
                                               y,
                                               z,
                                               r_over,
                                               sr.reshape(sr.size),
                                               xyzAntI.reshape(xyzAntI.size),
                                               xyzAntJ.reshape(xyzAntJ.size),
                                               focusedImg_a.reshape(focusedImg_a.size),
                                               myParameters)

elapsed = time.time() - t
print("execution time = " + str(elapsed))

focusedImg_a.shape = (1, 701, 201)
execution time = 7.494150161743164


## Plot focused images

In [43]:
cmap='jet'

### focusedImg_a

#### yz

In [44]:
toPlot = 20 * np.log10( np.abs( focusedImg_a[idx0_x,:,:].T ) )

fig, ax = plt.subplots(1,1)
im = ax.pcolor(y, z, toPlot, cmap=cmap)
dia.addColorBar( im, ax, aspect="equal" )

In [45]:
toPlot = 20 * np.log10( np.abs( focusedImg_a[idx0_x,:,:].T ) )

fig, ax = plt.subplots(1,1)
im = ax.imshow(toPlot, origin='lower', cmap=cmap, extent=extent_yz)
dia.addColorBar( im, ax, aspect="equal" )

In [46]:
from mpl_toolkits.axes_grid1 import AxesGrid

In [47]:
fig = plt.figure()
grid = AxesGrid(fig, 111,  # similar to subplot(111)
                nrows_ncols=(2, 2),
                axes_pad=0.0,
                share_all=True,
                label_mode="L",
                cbar_location="top",
                cbar_mode="single",
               )

for i in range(4):
    im = grid[i].imshow(toPlot, cmap=cmap, extent = extent_yx)
    #plt.colorbar(im, cax = grid.cbar_axes[0])
grid.cbar_axes[0].colorbar(im)

for cax in grid.cbar_axes:
    cax.toggle_label(False)


/home/pleroy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Attempting to set identical bottom == top == -0.2 results in singular transformations; automatically expanding.
  if sys.path[0] == '':


In [48]:
toPlot = 20 * np.log10( np.abs( focusedImg_a[50,:,:].T ) )

fig, ax = plt.subplots(1,1)
im = ax.pcolor(y, z, toPlot, cmap=cmap)
dia.addColorBar( im, ax, aspect="auto" )

IndexError: index 50 is out of bounds for axis 0 with size 1

#### yx

In [49]:
toPlot = 20 * np.log10( np.abs( focusedImg_a[:,:,0] ) )

fig, ax = plt.subplots(1,1)
im = ax.imshow(toPlot, origin='lower', cmap=cmap, extent=extent_yx)
dia.addColorBar( im, ax, aspect="equal" )
title = date + f" {ij}, delay = {d_shift}"
ax.set_title(title)

/home/pleroy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Attempting to set identical bottom == top == -0.2 results in singular transformations; automatically expanding.
  after removing the cwd from sys.path.


Text(0.5, 1.0, '2019_03_04 41, delay = 0.2')

In [50]:
opt = disp.OPTt()
opt.db = 1
opt.med_dyn = 100
opt.title = f"Synthesized image: hRail = {z1}, Rx = {Rx}, Tx = {Tx}\nground range, dB  d_gr={dy}, d_az={dx}"
disp.disp_PoSAR_img( focusedImg_a[:,:,0], opt, 0, 0, [yMin,yMax,xMax,xMin] ) # left, right, bottom, top = extent
ax = plt.gca()
ax.invert_xaxis()
ax.set_aspect("equal")
ax.yaxis.set_ticks_position("left")

db 1
mod 0
med_dyn 100
cmap jet
title Synthesized image: hRail = 3.41, Rx = 4, Tx = 1
ground range, dB  d_gr=0.01, d_az=0.01


/home/pleroy/DEV/processing/focalization_python/posarutils/process/disp_PoSAR_img.py:46: UserWarning: Attempting to set identical bottom == top == -0.2 results in singular transformations; automatically expanding.
  plt.imshow( disp_im, extent=im_extent, cmap=cmap )


# Process all

In [51]:
lib.reload()

## Define geometry

In [52]:
hScene = -1

dx = 0.01
dy = 0.01
dz = 0.01
xMin = -0.2
xMax = -0.2
yMin = 0
yMax = 7
zMin = -0.5
zMax = 1.5

imgGeom = (xMin, xMax, dx, yMin, yMax, dy, zMin, zMax, dz)

x = np.arange(xMin, xMax+1e-9, dx)
y = np.arange(yMin, yMax+1e-9, dy)
z = np.arange(zMin, zMax+1e-9, dz)
idx0_x = int( np.size(x)/2 )
idx0_y = int( np.size(y)/2 )
idx0_z = int( np.size(z)/2 )
nbX = x.size
nbY = y.size
nbZ = z.size

# lefg right bottom top
extent_yz = [yMin, yMax, zMin, zMax]
extent_yx = [yMin, yMax, xMin, xMax]

In [53]:
x_biz_offset = 0
y_biz_offset = 6.40
z_biz_offset = 3
biz_offset = (x_biz_offset, y_biz_offset, z_biz_offset)

In [54]:
myParameters           = MyParametersPoSAR_GB()
myParameters.Nx        = nbX
myParameters.Ny        = nbY
myParameters.Nz        = nbZ
myParameters.phi_a_deg = 60 # 10° => 4.3cm

In [55]:
overSamplingRatio = 10

img_h = {}

print( f"focusedImg.shape = ({nbX}, {nbY}, {nbZ})")

for data_idx in range(1, nbDir+1):
    
    path_xxx = base_path + f"/{prefix}_{data_idx}"
    z1 = heights[f"{data_idx}"]
    xyz_dic = gb.buildPosUsingTrack( path_xxx + "/track_S" , Dx1, Dx2, h, z1, verbose=1, bizona=biz_offset )
    print( f"z1 = {z1}" )
    
    img_ij = {}
    
    for ij in RxTx:

        Rx, Tx = ij[0], ij[1]
        xyzRx = xyz_dic[ Rx ]
        xyzTx = xyz_dic[ Tx ]
        hAver = ( xyzRx[0,i_z] + xyzTx[0,i_z] ) / 2
    
        read = gb.readData( path_xxx, f"S{ij}.dat" )
        
        sr = np.fft.fftshift( read[i_RD][:,0:read[i_CFG]["Nf"]-1], axes=1 )
        rangeResolution = read[i_CFG]["delta_sr"]
        Naz = sr.shape[0]
        Nf  = sr.shape[1]
    
        Nover   = overSamplingRatio * Nf
        shiftI = delays[ij[0]]
        shiftJ = delays[ij[1]]
        d_shift = shiftI + shiftJ
        r_base  = np.arange( Nf ) * rangeResolution - d_shift
        r_over  = np.arange( Nover ) * rangeResolution / overSamplingRatio - d_shift
        dr_over = r_over[1] - r_over[0]
    
        myParameters.Nover     = r_over.size
        myParameters.dx        = dr_over
        myParameters.Naz       = Naz
        myParameters.Nf        = Nf
        myParameters.kc        = read[i_CFG]["kc"]
    
        t = time.time()
    
        focusedImg  = np.zeros( (nbX, nbY, nbZ), dtype=complex )
        
        lib.so.backProjectionOmpGroundRange_PoSAR_GB_a( x,
                                                       y,
                                                       z,
                                                       r_over,
                                                       sr.reshape(sr.size),
                                                       xyzRx.reshape(xyzRx.size),
                                                       xyzTx.reshape(xyzTx.size),
                                                       focusedImg.reshape(focusedImg.size),
                                                       myParameters)

        elapsed = time.time() - t
        print( f"hAver = {hAver:.3f}, (Rx,Tx) = ({Rx},{Tx}), execution time = {elapsed:.1f}, delay = {d_shift:.3f}")
        img_ij[ij] = focusedImg
    img_h[z1] = img_ij

focusedImg.shape = (1, 701, 201)
read 500 positions in file:
/home/pleroy/DATA/DIADEM/lff/Bizona/2019_03_04/sand_trunk_vol_1/track_S21.dat
z1 = 3.41
hAver = 3.470, (Rx,Tx) = (2,1), execution time = 7.4, delay = 0.200
hAver = 3.450, (Rx,Tx) = (3,1), execution time = 7.3, delay = 0.200
hAver = 3.430, (Rx,Tx) = (3,2), execution time = 7.3, delay = 0.200
hAver = 3.245, (Rx,Tx) = (4,1), execution time = 7.4, delay = 0.200
hAver = 3.225, (Rx,Tx) = (4,2), execution time = 7.5, delay = 0.200
hAver = 3.205, (Rx,Tx) = (4,3), execution time = 7.3, delay = 0.200
read 500 positions in file:
/home/pleroy/DATA/DIADEM/lff/Bizona/2019_03_04/sand_trunk_vol_2/track_S21.dat
z1 = 3.33
hAver = 3.390, (Rx,Tx) = (2,1), execution time = 7.4, delay = 0.200
hAver = 3.370, (Rx,Tx) = (3,1), execution time = 7.3, delay = 0.200
hAver = 3.350, (Rx,Tx) = (3,2), execution time = 7.3, delay = 0.200
hAver = 3.205, (Rx,Tx) = (4,1), execution time = 7.4, delay = 0.200
hAver = 3.185, (Rx,Tx) = (4,2), execution time = 7.3, d

## save images

In [ ]:
if z.size == 1:
    for idx in heights:
        hRef = heights[idx]
        print(hRef)
        with open(base_path + f"/{date}_hRef{hRef:.3f}_z{zMin:.3f}.dat", 'wb') as file:
            pk.dump((imgGeom, img_h[hRef]), file)
if x.size == 1:
    try:
        newDir = base_path + f"/x{xMin:.3f}"
        # Create target Directory
        os.mkdir(newDir)
        print( "Directory " , newDir, " Created " ) 
    except FileExistsError:
        print( "Directory " , newDir, " already exists" )
    for idx in heights:
        hRef = heights[idx]
        print(hRef)
        filename = newDir + f"/hRef{hRef:.3f}-{x_biz_offset:.2f}-{y_biz_offset:.2f}-{z_biz_offset:.2f}.dat"
        with open( filename, 'wb' ) as file:
            pk.dump((imgGeom, img_h[hRef]), file)
else:
    for idx in heights:
        hRef = heights[idx]
        print(hRef)
        with open(base_path + f"/{date}_hRef{hRef:.3f}_x{xMin:.3f}_{xMax:.3f}.dat", 'wb') as file:
            pk.dump((imgGeom, img_h[hRef]), file)

# Process all tomo 3D

## Define geometry

In [ ]:
hScene = -1

dx = 0.01
dy = 0.01
dz = 0.01
xMin = -1.5
xMax = 1.5
yMin = 0
yMax = 7
zMin = 0.0
zMax = 0.0

imgGeom = (xMin, xMax, dx, yMin, yMax, dy, zMin, zMax, dz)

x = np.arange(xMin, xMax+1e-9, dx)
y = np.arange(yMin, yMax+1e-9, dy)
z = np.arange(zMin, zMax+1e-9, dz)
idx0_x = int( np.size(x)/2 )
idx0_y = int( np.size(y)/2 )
idx0_z = int( np.size(z)/2 )
nbX = x.size
nbY = y.size
nbZ = z.size

# lefg right bottom top
extent_yz = [yMin, yMax, zMin, zMax]
extent_yx = [yMin, yMax, xMin, xMax]

## Focusing

In [ ]:
myParameters           = MyParametersPoSAR_GB()
myParameters.Nx        = nbX
myParameters.Ny        = nbY
myParameters.Nz        = nbZ
myParameters.phi_a_deg = 60 # 10° => 4.3cm

In [ ]:
overSamplingRatio = 10

img_h = {}

imgMono  = np.zeros((nbX, nbY, nbZ), dtype=complex)
imgBi    = np.zeros((nbX, nbY, nbZ), dtype=complex)
averMono = 0
averBi   = 0

print( f"focusedImg.shape = ({nbX}, {nbY}, {nbZ})")

for data_idx in range(1, nbDir+1):
    
    path_xxx = base_path + f"/{prefix}_{data_idx}"
    z3 = heights[f"{data_idx}"]
    xyz_dic = gb.buildPosUsingTrack( path_xxx + "/track_S" , Dx1, Dx2, h, z3, verbose=1, bizona=biz_offset )
    print( f"z3 = {z3}" )
    
    img_ij = {}
    
    for ij in RxTx:

        Rx, Tx = ij[0], ij[1]
        xyzRx = xyz_dic[ Rx ]
        xyzTx = xyz_dic[ Tx ]
        hAver = ( xyzRx[0,i_z] + xyzTx[0,i_z] ) / 2
    
        read = gb.readData( path_xxx, f"S{ij}.dat" )
        
        sr = np.fft.fftshift( read[i_RD][:,0:read[i_CFG]["Nf"]-1], axes=1 )
        rangeResolution = read[i_CFG]["delta_sr"]
        Naz = sr.shape[0]
        Nf  = sr.shape[1]
    
        Nover   = overSamplingRatio * Nf
        shiftI = delays[ij[0]]
        shiftJ = delays[ij[1]]
        d_shift = shiftI + shiftJ
        r_base  = np.arange( Nf ) * rangeResolution - d_shift
        r_over  = np.arange( Nover ) * rangeResolution / overSamplingRatio - d_shift
        dr_over = r_over[1] - r_over[0]
    
        myParameters.Nover     = r_over.size
        myParameters.dx        = dr_over
        myParameters.Naz       = Naz
        myParameters.Nf        = Nf
        myParameters.kc        = read[i_CFG]["kc"]
    
        t = time.time()
    
        focusedImg  = np.zeros( (nbX, nbY, nbZ), dtype=complex )
        
        lib.so.backProjectionOmpGroundRange_PoSAR_GB_a( x,
                                                       y,
                                                       z,
                                                       r_over,
                                                       sr.reshape(sr.size),
                                                       xyzRx.reshape(xyzRx.size),
                                                       xyzTx.reshape(xyzTx.size),
                                                       focusedImg.reshape(focusedImg.size),
                                                       myParameters)

        elapsed = time.time() - t
        print( f"hAver = {hAver:.3f}, (Rx,Tx) = ({Rx},{Tx}), execution time = {elapsed:.1f}, delay = {d_shift:.3f}")
        if ij in ["21", "31", "32"]:
            imgMono  += focusedImg
            averMono += 1
        if ij in ["41", "42", "43"]:
            if z3 != heights['1']:
                if ij != "41":
                    imgBi  += focusedImg
                    averBi += 1
            else:
                imgBi  += focusedImg
                averBi += 1
imgBi   /= averBi
imgMono /= averMono

In [ ]:
with open(base_path + f"/{date}_3D_mono.dat", 'wb') as file:
    pk.dump((imgGeom, imgMono), file)
with open(base_path + f"/{date}_3D_bi.dat", 'wb') as file:
    pk.dump((imgGeom, imgBi), file)

In [ ]:
with open(base_path + f"/{date}_3D_mono_{imgMono.dtype}.bin", 'wb') as file:
    imgMono.tofile( file )
with open(base_path + f"/{date}_3D_bi_{imgBi.dtype}.bin", 'wb') as file:
    imgBi.tofile( file )

## Plot tomo mono

In [ ]:
toPlot = 20 * np.log10( np.abs( np.squeeze(imgMono) ).T )
fig, ax = plt.subplots(1,1)
im = ax.imshow(toPlot, cmap='jet', origin='lower', extent=extent_yz)
dia.addColorBar( im, ax, aspect="equal" )
title = f"{date} {prefix}\nmonostatic x = {x[idx0_x]:.2f}"
ax.set_title(title)

## Plot tomo bi

In [ ]:
toPlot = 20 * np.log10( np.abs( np.squeeze( imgBi ) ).T )
fig, ax = plt.subplots(1,1)
im = ax.imshow(toPlot, cmap='jet', origin='lower', extent=extent_yz )
dia.addColorBar( im, ax, aspect="equal" )
title = f"{date} {prefix}\nbistatic x = {x[idx0_x]:.2f}"
ax.set_title(title)